In [ ]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your googlbie drive.
from google.colab import drive
drive.mount('/gdrive')
root = '/gdrive/My Drive/CS492I/project'

Mounted at /gdrive


## 1. Collect CodeSearchNet Repositories

In [ ]:
import json

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!mkdir CodeSearchNet
!unzip python.zip -d CodeSearchNet

--2021-12-03 07:32:59--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.69.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.69.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  65.4MB/s    in 13s     

2021-12-03 07:33:12 (68.1 MB/s) - ‘python.zip’ saved [940909997/940909997]

Archive:  python.zip
   creating: CodeSearchNet/python/
   creating: CodeSearchNet/python/final/
   creating: CodeSearchNet/python/final/jsonl/
   creating: CodeSearchNet/python/final/jsonl/train/
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/pyt

In [ ]:
python_files = sorted(Path('CodeSearchNet/python').glob('**/*.gz'))
print(python_files)

[PosixPath('CodeSearchNet/python/final/jsonl/test/python_test_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_1.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_11.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_13.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_2.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_3.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_4.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_5.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_6.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_7.jsonl.gz

In [ ]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [ ]:
pydf = jsonl_list_to_dataframe(python_files, columns=['repo'])
pydf = pydf.drop_duplicates().reset_index(drop=True)
pydf.to_pickle(f"{root}/repos.pkl")

## 2. Collect diff and commits

In [57]:
!pip install pydriller
!pip install pandas
!pip install spacy

In [74]:
from pydriller import *
import pandas as pd
from nltk.tokenize import regexp_tokenize
import spacy
import re
from itertools import chain
import json

In [59]:
repodf = pd.read_pickle(f"{root}/repos.pkl")
print(repodf.shape)
spacy_tokenizer = spacy.load("en_core_web_sm")
print(repodf)

(13590, 1)


Exception ignored in: <finalize object at 0x7f50870debe0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpkjuvkjmw'


                                 repo
0                     soimort/you-get
1                      apache/airflow
2                      pytorch/vision
3                      asciimoo/searx
4              tensorflow/probability
...                               ...
13585         praekelt/python-ambient
13586                 zenreach/py-era
13587  TakesxiSximada/custom_settings
13588            openpermissions/bass
13589               xnuinside/clifier

[13590 rows x 1 columns]


In [86]:
def basic_filter(message):
    return message.split("\n", 1)[0].strip()

# Remove [label] in front of commit if exists
def label_filter(message):
    if (message.startswith('[')):
        end_bracket_index = message.find(']')
        if (end_bracket_index == -1):
            return None
        return message[:end_bracket_index+1]
    return message

def camel_case_split(str):
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

def case_splitter(token):
    return list(map(lambda x: x.lower(), camel_case_split(token[0].upper() + token[1:]) if token[0].isalpha() else [token]))


def split_by_quote(diff):
    text_list = [(0,0)]
    quote_state = 0 #["out","single","double","Single","Double","quote"]
    i=0
    while i<len(diff):
      if diff[i] == "\\":i+=1
      elif (diff[i] == "\n" and quote_state == 5) or (diff[i] == '#' and quote_state == 0):
        text_list.append((quote_state,i))
        quote_state = abs(5-quote_state)
      elif diff[i] == '"':
        if i+2<len(diff) and diff[i:i+3]=='"""' and (quote_state == 0 or quote_state ==4):
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(4-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 2:
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(2-quote_state)
      elif diff[i] == "'":
        if i+2<len(diff) and diff[i:i+3] == "'''" and (quote_state == 0 or quote_state == 3):
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(3-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 1:
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(1-quote_state)
      i+=1
    text_list.append((quote_state,i))
    return [(text_list[i][0]==0,diff[text_list[i-1][1]:text_list[i][1]]) for i in range(1,len(text_list))]
  

token_regex = r"""(?x)
<(?:add|del)>   #Filtered eariler
|(?:[-+*/^&~|=%!<>@?$][\s]*)+     #Sequence of symbols
|[\n]                    #Change row
|[a-zA-Z]+               #General text
|[0-9]+                  #Number
"""
string_regex = r"""(?x)
<(?:add|del)>
|[\n]
|[a-zA-Z]+
"""
def diff_tokenizer(diff_text):
    diff = '\n'.join(map(lambda x: x[1], filter(lambda y: y[0] % 2 == 0, enumerate(diff_text.split("@@")))))
    diff = diff.replace('\n+', '\n<add>').replace('\n-', '\n<del>')
    diff = re.sub(r"(?:\n[ \t\r\f\v]*)+","\n",diff) # Join continuous row change
    quote_split = split_by_quote(diff)
    token_initial = chain.from_iterable(map(lambda a: regexp_tokenize(a[1],(token_regex if a[0] else string_regex)), quote_split))
    token_camel_case_split = chain.from_iterable(map(case_splitter, token_initial))
    return token_camel_case_split


def parse_repo_commits(repo_name, commit_limit=50):
    data = []
    commit_count = 0
    for commit in Repository(
        f"https://github.com/{repo_name}",
        only_modifications_with_file_types=[".py"],
        only_no_merge=True,
        order='reverse'
    ).traverse_commits():
        if (commit_count >= commit_limit): break
        line = basic_filter(commit.msg)
        line = label_filter(line)
        if (line is None):
            continue
        
        if line.count("`")%2 == 1:
            continue
        
        backtick_regex = r"`[^`]+`|[^`]+"
        line_divided_by_backtick = re.findall(backtick_regex,line)
        tokens = chain.from_iterable(map(lambda text: regexp_tokenize(text,(token_regex if text[0] == '`' else string_regex)),line_divided_by_backtick))
        
        commit_tokens = list(chain.from_iterable(map(case_splitter, tokens)))

        if (len(commit_tokens) < 3 or len(commit_tokens) > 30):
            continue

        # Check if changed files are python
        file_failed = False
        count_diff_lines = 0

        for mf in commit.modified_files:
            if (not mf.filename.endswith(".py")):
                file_failed = True
                break
            count_diff_lines += mf.added_lines + mf.deleted_lines
        
        if (file_failed):
            continue
        
        if (count_diff_lines > 50):
            continue
        
        # Create diff tokens
        diff_tokens = []
        for f in commit.modified_files:
          diff_tokens.append (['<file>'])
          diff_tokens.append(diff_tokenizer(f.diff))

        diff_whole_tokens = list(chain.from_iterable(diff_tokens))

        data.append([repo_name, commit.hash, json.dumps(commit_tokens), json.dumps(diff_whole_tokens)])
        commit_count += 1

    return pd.DataFrame(data, columns=["repo", "hash", "commit_messsage", "diff"])

In [87]:
import time
start = time.time()
#df = parse_repo_commits("soimort/you-get")
df = parse_repo_commits("tensorflow/probability")
end = time.time()
print(end - start)
df.head(10)

25.511131048202515


,repo,hash,commit_messsage,diff
0,tensorflow/probability,c2299ff88eaf1c3f29dc48ce5a80c3db8bc8625d,"[""ensure"", ""owens"", ""t"", ""special"", ""function"", ""gracefully"", ""handles"", ""partially"", ""dynamic"", ""shapes""]","[""<file>"", ""\n"", ""def"", ""owens"", ""t"", ""method"", ""1"", ""h"", ""a"", ""m"", ""\n"", ""should"", ""stop"", ""= "", ""index"", "">= "", ""m"", ""\n"", ""return"", ""should"", ""stop"", ""index"", ""+ "", ""1"", ""new"", ""ai"", ""new"", ""di"", ""new"", ""gi"", ""new"", ""series"", ""sum"", ""\n"", ""<del>"", ""broadcast"", ""shape"", ""= "", ""prefer"", ""static..."
1,tensorflow/probability,f5541be80d2e118a3875ec795285927f5580465a,"[""always"", ""return"", ""kalman"", ""filter"", ""results"", ""as"", ""a"", ""filter"", ""results"", ""namedtuple""]","[""<file>"", ""\n"", ""from"", ""tensorflow"", ""python"", ""ops"", ""import"", ""parallel"", ""for"", ""pylint"", ""disable"", ""g"", ""direct"", ""tenso"", ""\n"", ""tfl"", ""= "", ""tf"", ""linalg"", ""\n"", ""<add>"", ""filter"", ""results"", ""= "", ""collections"", ""namedtuple"", ""\n"", ""<add>"", ""filter"", ""results"", ""\n"", ""<add>"", ""log"", ""l..."
2,tensorflow/probability,9990d6105450b43e2169968a397c0ce8d98598cb,"[""improves"", ""error"", ""message"", ""in"", ""dtype"", ""util"", ""common"", ""dtype""]","[""<file>"", ""\n"", ""def"", ""base"", ""equal"", ""a"", ""b"", ""\n"", ""return"", ""base"", ""dtype"", ""a"", ""== "", ""base"", ""dtype"", ""b"", ""\n"", ""<del>"", ""def"", ""common"", ""dtype"", ""args"", ""list"", ""dtype"", ""hint"", ""="", ""none"", ""\n"", ""<del>"", ""returns"", ""explict"", ""dtype"", ""from"", ""args"", ""list"", ""if"", ""there"", ""is"", ..."
3,tensorflow/probability,ae7117f37ac441bc7a888167ea23e5e620c5bcde,"[""hack"", ""convert"", ""to"", ""nested"", ""tensor"", ""to"", ""forward"", ""keras"", ""tensors"", ""untouched""]","[""<file>"", ""\n"", ""class"", ""bijector"", ""caching"", ""test"", ""test"", ""util"", ""test"", ""case"", ""\n"", ""z"", ""= "", ""instance"", ""cache"", ""bijector"", ""forward"", ""x"", ""\n"", ""self"", ""assert"", ""is"", ""not"", ""y"", ""z"", ""\n"", ""<add>"", ""@"", ""test"", ""util"", ""jax"", ""disable"", ""test"", ""missing"", ""functionality"", ""ker..."
4,tensorflow/probability,b305c9c66ae1b7f3f2446eeb9422fcff18eeb0e3,"[""restrain"", ""hypothesis"", ""from"", ""generating"", ""absurd"", ""limit"", ""arguments"", ""to"", ""truncated"", ""normal""]","[""<file>"", ""\n"", ""def"", ""fix"", ""triangular"", ""d"", ""\n"", ""return"", ""dict"", ""d"", ""peak"", ""="", ""peak"", ""high"", ""="", ""high"", ""\n"", ""<add>"", ""def"", ""fix"", ""truncated"", ""normal"", ""params"", ""\n"", ""<add>"", ""new"", ""params"", ""= "", ""dict"", ""params"", ""high"", ""="", ""tfp"", ""hps"", ""ensure"", ""high"", ""gt"", ""low"",..."
5,tensorflow/probability,16c382e6ca2fd897c8023a4169eeab9b3575c134,"[""re"", ""narrow"", ""numerical"", ""tolerances"", ""perturbed"", ""by"", ""hypothesis"", ""upgrade"", ""from"", ""x""]","[""<file>"", ""\n"", ""autovectorization"", ""rtol"", ""update"", ""\n"", ""matvec"", ""lu"", ""1"", ""e"", ""-"", ""4"", ""todo"", ""b"", ""tighten"", ""this"", ""\n"", ""scale"", ""matvec"", ""lu"", ""1"", ""e"", ""-"", ""2"", ""todo"", ""b"", ""tighten"", ""this"", ""\n"", ""scale"", ""matvec"", ""tri"", ""l"", ""1"", ""e"", ""-"", ""3"", ""todo"", ""b"", ""tighten"", ""t..."
6,tensorflow/probability,8e03d0ce95154f0e4b495c69ed21e7b58641d705,"[""relax"", ""precision"", ""of"", ""truncated"", ""normal"", ""in"", ""hypothesis"", ""test""]","[""<file>"", ""\n"", ""event"", ""space"", ""bijector"", ""is"", ""broken"", ""= "", ""\n"", ""slicing"", ""logprob"", ""atol"", ""= "", ""collections"", ""defaultdict"", ""lambda"", ""1"", ""e"", ""-"", ""5"", ""\n"", ""slicing"", ""logprob"", ""atol"", ""update"", ""\n"", ""normal"", ""inverse"", ""gaussian"", ""3"", ""e"", ""-"", ""4"", ""b"", ""\n"", ""<add>"", ..."
7,tensorflow/probability,40b18076b53d7c6e22e1fa5f95d805be

# 3. Create whole dataset via Multiprocessing

In [ ]:
import multiprocessing

data = []

def f(repo):
    start = time.time()
    df = parse_repo_commits(repo)
    #df.to_pickle(f"./repos/{repo.replace('/', '+')}.pkl")
    end = time.time()
    print(f"{repo} took {end-start} seconds")
    return df

pool = multiprocessing.Pool()
outputs = pool.map(f, repodf['repo'][:3])
pd.concat(outputs).to_pickle(f"{root}/data.pkl")

In [ ]:
df = pd.read_pickle("data.pkl")
df.head(3)